## Import Datasets and Libraries

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
# Load dataset
data = fetch_california_housing()
X_np = data.data  # shape: (20640, 8)
y_np = data.target.reshape(-1, 1)  # shape: (20640, 1)

# Scale features (important for gradient descent)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_np)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_np, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
score = r2_score(y_pred, y_test)
print('Score: ', score)
print(model.coef_)
print(model.intercept_)

Score:  0.3376701658930997
[[ 0.85238169  0.12238224 -0.30511591  0.37113188 -0.00229841 -0.03662363
  -0.89663505 -0.86892682]]
[2.06786231]


## Model Using Neural Networks (PyTorch):
### No Hidden Layers

In [3]:
import torch
import torch.nn as nn
#Convert to Tensors
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [4]:
model = nn.Linear(in_features=8, out_features=1)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
n_epochs = 1000

for epoch in range(n_epochs):
    model.train()
    y_pred = model(X_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Training Loss = {loss.item():.4f}")

Epoch 0: Training Loss = 4.8135
Epoch 100: Training Loss = 1.4749
Epoch 200: Training Loss = 0.6832
Epoch 300: Training Loss = 0.5352
Epoch 400: Training Loss = 0.5190
Epoch 500: Training Loss = 0.5180
Epoch 600: Training Loss = 0.5179
Epoch 700: Training Loss = 0.5179
Epoch 800: Training Loss = 0.5179
Epoch 900: Training Loss = 0.5179


In [5]:
def r2_score_torch(y_true, y_pred):
    ss_total = ((y_true - y_true.mean()) ** 2).sum()
    ss_res = ((y_true - y_pred) ** 2).sum()
    r2 = 1 - ss_res / ss_total
    return r2.item()
    
with torch.no_grad():
    y_test_pred = model(X_test_tensor)
    score = r2_score_torch(y_test_pred, y_test_tensor)

print(f"R2 score: {score:.4f}")
print(model.weight.data)
print(model.bias.data)

R2 score: 0.3377
tensor([[ 0.8524,  0.1224, -0.3051,  0.3711, -0.0023, -0.0366, -0.8966, -0.8689]])
tensor([2.0679])


### One Hidden Layer With ReLU

In [6]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initializing the model
input_dim = X_train.shape[1]
hidden_dim = 128  # can try other values like 32 or 128
model = SimpleNN(input_dim, hidden_dim)

In [7]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
n_epochs = 1000

for epoch in range(n_epochs):
    model.train()
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Training Loss = {loss.item():.4f}")


Epoch 0: Training Loss = 6.2244
Epoch 100: Training Loss = 0.3850
Epoch 200: Training Loss = 0.3403
Epoch 300: Training Loss = 0.3107
Epoch 400: Training Loss = 0.2941
Epoch 500: Training Loss = 0.2842
Epoch 600: Training Loss = 0.2770
Epoch 700: Training Loss = 0.2708
Epoch 800: Training Loss = 0.2650
Epoch 900: Training Loss = 0.2603


In [8]:
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor)
    r2 = r2_score_torch(y_test_tensor, y_test_pred)
    print("R² Score on Test Set:", r2)

R² Score on Test Set: 0.7826993465423584


### Two Hidden Layers with ReLU

In [9]:
class TwoLayerNN(nn.Module):
    def __init__(self, input_dim, hidden1_dim, hidden2_dim):
        super(TwoLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden1_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_dim, hidden2_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_dim, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Example architecture
input_dim = X_train.shape[1]  # 8
hidden1_dim = 64
hidden2_dim = 32

model = TwoLayerNN(input_dim, hidden1_dim, hidden2_dim)

In [12]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
n_epochs = 1000

for epoch in range(n_epochs):
    model.train()
    y_pred = model(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Training Loss = {loss.item():.4f}")

Epoch 0: Training Loss = 0.2815
Epoch 100: Training Loss = 0.2745
Epoch 200: Training Loss = 0.2673
Epoch 300: Training Loss = 0.2620
Epoch 400: Training Loss = 0.2570
Epoch 500: Training Loss = 0.2512
Epoch 600: Training Loss = 0.2465
Epoch 700: Training Loss = 0.2429
Epoch 800: Training Loss = 0.2396
Epoch 900: Training Loss = 0.2408


In [13]:
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor)
    r2 = r2_score_torch(y_test_tensor, y_test_pred)
    print("R² Score on Test Set:", r2)

R² Score on Test Set: 0.7905864715576172
